## 모듈 불러오기

In [32]:
from pykrx import stock
import pandas as pd
import numpy as np
from pandas import Series
from tqdm import tqdm
from tqdm import notebook
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib as mpl
import time


In [33]:
import os
os.getcwd()

'C:\\Users\\yukino\\Downloads\\KQR\\페이즈4'

# 머신러닝, 딥러닝 모듈 불러오기

In [96]:
# 딥러닝 모듈
from keras.models import load_model

deep_model = load_model('realdeepmodel.h5')


 #머신러닝 모듈
#import joblib

#ml_model = joblib.load('svc.pkl')


## 백테스팅을 위한 X데이터 불러오기 (X데이터는 딥러닝 예측 모델의 구현에 이용된다.)

## 여기서 사용되는 X데이터는 2600 * 9 개의 행으로 된 결측치 처리가 안된 X데이터를 이용한다.
## 13개의 데이터를 나누기 용이하게 하기 위함


In [97]:

X_data = pd.read_csv("RawDataset_train.csv") #       RawDataset_backtest
X_data = X_data.set_index("Unnamed: 0")
X_data

,PER,EPS증가율,저변동성,QUANTITY,PER스코어,EPS스코어,VOL스코어,QUT스코어,y
Unnamed: 0,,,,,,,,,
A196170,NaN,-80.0,0.023210,0.004336,NaN,0.495120,-0.146157,0.015120,0
A036490,18.63,-6.2,0.021455,-0.000799,-0.252433,0.178809,-0.325200,0.086172,0
A039490,8.06,33.5,0.021396,0.095315,-0.345448,0.008652,-0.331301,-1.243634,0
A088350,10.26,-26.5,0.016679,-0.065330,-0.326088,0.265816,-0.812486,0.979001,0
A112610,11.61,0.5,0.024002,0.011859,-0.314208,0.150092,-0.065434,-0.088954,0
...,...,...,...,...,...,...,...,...,...
A183190,11.90,-14.3,0.026477,-0.053144,-0.138595,0.079423,-0.860322,0.681420,0
A067000,40.44,0.5,0.047368,-0.020614,-0.089366,0.071533,0.615191,0.054671,0
A950160,NaN,-80.0,0.000000,NaN,NaN,0.114451,-2.730486,NaN,0


# 백테스트 함수

## 보조지표 함수-칼럼추가형식

### 이동평균선

In [5]:
#단순 이동 평균(Simple Moving Average, SMA)
def SMA(data, period=30, column='종가') :
    return data[column].rolling(window=period).mean()


#지수 이동 평균(Exponential Moving Average, EMA)
def EMA(data, period=20, column='종가') :
    return data[column].ewm(span=period, adjust=False).mean()

### MACD

In [6]:
def MACD(data, period_long=26, period_short=12, period_signal=9, column='종가'):
    #단기 지수 이평선 계산
    ShortEMA = EMA(data, period_short, column=column)
    
    #장기 지수 이평선 계산
    LongEMA = EMA(data, period_long, column=column)
    
    #이동 평균 수렴/발산 계산
    data['MACD'] = ShortEMA - LongEMA
    
    #신호선 계산
    data['Signal_Line'] = EMA(data, period_signal, column='MACD')
    
    return data

### RSI

In [7]:
#상대적 강도 지수 계산

def RSI(data, period=14, column='종가'):
    delta = data[column].diff(1)
    delta = delta.dropna()
    
    up = delta.copy()
    down = delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    data['up'] = up
    data['down'] = down
    
    AVG_Gain = SMA(data, period, column = 'up')
    AVG_Loss = abs(SMA(data, period, column='down'))
    RS = AVG_Gain / AVG_Loss
    
    RSI = 100.0 - (100.0 / (1.0 + RS))
    data['RSI'] = RSI
    
    return data

### DMI

In [8]:
# +DM

def DM(data, high = '고가', low = '저가') :
    
    data['HM'] = data[high] - data.shift(1)[high]
    data['LM'] = data.shift(1)[low] - data[low]
    #data['MM'] = data['HM'] - data['LM']
    #data = data.dropna()
    
    data['PDM'] = data['HM'][(data['HM'] >0) & (data['HM'] >data['LM'])]
    data['MDM'] = data['LM'][(data['LM'] >0) & (data['HM'] <data['LM'])]
    
    data['PDM'] = data['PDM'].fillna(0)
    data['MDM'] = data['MDM'].fillna(0)

    return data


def TR(data, high = '고가', low = '저가', col='종가') :
    
    data['TR1'] = abs(data[high] - data[low])
    data['TR2'] = abs(data[high] - data.shift(1)[col])
    data['TR3'] = abs(data[low] - data.shift(1)[col])
    
    
    data['TRa'] = data['TR1'][ (data['TR1']>=data['TR2']) & (data['TR1']>=data['TR3'])]
    data['TRb'] = data['TR2'][ (data['TR2']>=data['TR1']) & (data['TR2']>=data['TR3'])]
    data['TRc'] = data['TR3'][ (data['TR3']>=data['TR1']) & (data['TR3']>=data['TR2'])]
    
    data['TRa']= data['TRa'].fillna(0)
    data['TRb']= data['TRb'].fillna(0)
    data['TRc']= data['TRc'].fillna(0)
    
    data['TR'] = data['TRa']+data['TRb']+data['TRc']
        
        
    return data


def DI(data) :
    
    data['PMI'] = (data['PDM'].rolling(window=14).mean()) / (data['TR'].rolling(window=14).mean())
    data['DMI'] = (data['MDM'].rolling(window=14).mean()) / (data['TR'].rolling(window=14).mean())

    return data


### ADX

In [9]:
def ADX(data, period = 14) :
    
    data['DX'] = abs(data['PMI'] - data['DMI']) / (data['PMI'] + data['DMI'])
    
    data['ADX'] = data['DX'].ewm(span=period, adjust=False).mean()
    
    return data

### CCI

In [10]:
def CCI(data, period = 14) :
    
    data['M_sum'] = (data['고가'] + data['저가'] + data['종가'])/3
    
    data['N_mov'] = data['M_sum'].rolling(window=period).mean()
    
    data['D'] = abs(data['M_sum']-data['N_mov']).rolling(window=period).mean()
    
    data['CCI'] = (data['M_sum']-data['N_mov'])/(data['D']*0.015)
    
    return data

### Stochastic Slow

In [11]:
def Stochastic(data , N = 5, M =3 ,T=3) :
    
    data['fastK%'] = ((data['종가'] - data['종가'].rolling(window=N).min())/(data['종가'].rolling(window=N).max() -
                                                                       data['종가'].rolling(window=N).min())) * 100
    
    data['fastD%'] = data['fastK%'].ewm(span=M, adjust=False).mean()
    
    data['slowK%'] = data['fastK%'].ewm(span=T, adjust=False).mean()
    
    data['slowD%'] = data['fastD%'].ewm(span=T, adjust=False).mean()

    return data

### Gradient function-local extremum point(극값얻기)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from sympy import *

def Gradient(data, column = '종가', period = 10) :
    
    number = np.array(list(range(period)))[:,np.newaxis]

    splited = []
    
    gradient = []
    
    for i in range(len(data[column])- period +1):
            
        splited.append(data[column][i:i+period])
            
    
    for num ,j in enumerate(splited) :
        

        pr = LinearRegression()
        
        quadratic  = PolynomialFeatures(degree=2)
        
        X_quad = quadratic.fit_transform(number)
        
        pr.fit(X_quad,np.array(j))
            
        coef_a = pr.coef_[2]*2
        coef_b = pr.coef_[1]
        
        x = Symbol('x')
        
        f = coef_a * x + coef_b
        
        critical = solve(f)
        
        if (critical[0] >= 0) & (critical[0] <= (len(splited)-1)) & (coef_a<0):
            
            gradient.append("극대")
            
        elif (critical[0] >= 0) & (critical[0] <= (len(splited)-1)) & (coef_a>0):
            
            gradient.append("극소")
            
        else :
            
            gradient.append("Nan")

        
    
    for k in range(period-1) :
        gradient.insert(0,"Nan")

    
    data['Gradient'] = gradient
        
        
    return data
    
    
    


## 백테스트 함수

In [13]:
def backtest(start_date,end_date,code,start_money):
    

    
    if start_date == '20180101' :
        
        start_date  = '20171211'
        
    elif start_date == '20180401':
        
        start_date = '20180314'
        
    elif start_date == '20180701':
        
        start_date = '20180613'
        
    elif start_date == '20181001':
        
        start_date = '20180913'
        
    elif start_date == '20190101':
        
        start_date = '20181211'
        
    elif start_date == '20190401':
        
        start_date = '20190314'
        
    elif start_date == '20190701':
        
        start_date = '20190613'
        
    elif start_date == '20191001':
        
        start_date = '20190913'
        
    elif start_date == '20200101':
        
        start_date = '20191211'
        
    elif start_date == '20200401':
        
        start_date = '20200314'
        
    elif start_date == '20200701':
        
        start_date = '20200613'
        
    elif start_date == '20201001':
        
        start_date = '20200913'
        
    elif start_date == '20210101':
        
        start_date = '20201211'

        

     
    
    
    # 주가 정보 가져오기
    df = stock.get_market_ohlcv_by_date(start_date,end_date,code)
    name = stock.get_market_ticker_name(code)

    # 초기설정
    start = start_money
    start_money =start_money
    high = start_money #전고점
    
    #
    slipage = 0.03

    max_bench = 0
    mdd_bench = 0
    
    df = MACD(df, period_long=13, period_short=6, period_signal=4)
    
    #MACD 매수 매도 시그널 -양의 방향으로 움직이는 macd일때
    
    df['MACD매수'] = (df['MACD'] > df['Signal_Line']) & (df['MACD'] > 0) & ((df['MACD'] - df['Signal_Line']) > 
                                                                          (df.shift(1)['MACD'] - df.shift(1)['Signal_Line']))
    
    df['MACD매도'] = (df['MACD'] < df['Signal_Line']) & (df['MACD'] < 0) & ((df['MACD'] - df['Signal_Line']) < 
                                                                          (df.shift(1)['MACD'] - df.shift(1)['Signal_Line']))
    
    
    # DMI 방향운동지수 매수 매도 시그널
    
    df = DM(df)
    df = TR(df)
    df = DI(df)
    
    df['DI매수'] = (df['PMI'] > df['DMI']) & ((df['MACD'] - df['Signal_Line']) > (df.shift(1)['MACD'] -
                                                                                df.shift(1)['Signal_Line']))&(df['MACD'] > 0)

    df['DI매도'] =  (df['PMI'] < df['DMI']) & ((df['MACD'] - df['Signal_Line']) < (df.shift(1)['MACD'] -
                                                                                df.shift(1)['Signal_Line']))&(df['MACD'] < 0)   
    
    df= df[13:]
    


    
    # backtest LIST
    list1 = list(df['종가'])
    list2 = list(df['MACD매수'])
    list3 = list(df['MACD매도'])
    list4 = list(df['DI매수'])    
    list5 = list(df['DI매도'])
    
    
    #이것만 맞춰주면 되잖아
    asset_list = []
    profit_list = []
    profit_percent_list = []
    maximum_list = []
    mdd_list = []
    
    #딕셔너리 설정
    box = {"보유여부" : 0 , "보유가격" : 0, "보유개수" : 0 }

    rest = 0 #남은 현금
    
        #매수매도 부분 
        
    for i,j in enumerate(list4):
        
        
        if box["보유여부"] == 0 :
            
            if j == True :
                
                box["보유여부"] = 1
                box["보유가격"] = list1[i]
                box["보유개수"] = start_money//list1[i]
                
                rest = start_money - list1[i]*box["보유개수"]
                
                asset = start_money
                profit = 0
                maximum = asset
                profit_percent =0
                
            else : 
                asset = start_money
                profit = 0
                maximum = start_money
                profit_percent =0
                
        else : #보유중인경우
            
            if list5[i] == True :
                
                asset = list1[i]*box["보유개수"] + rest
                profit = (list1[i]-box["보유가격"])*box["보유개수"]
                maximum = max(max(asset_list),asset)
                profit_percent = 100*(list1[i]/box["보유가격"] -1) #수익률

                asset_list.append(asset)
                profit_list.append(profit)
                maximum_list.append(maximum)
                profit_percent_list.append(profit_percent)
                
                
                if (asset - maximum) > 0:
                    mdd = 0
                else:
                    mdd = ((asset-maximum)/maximum)*100
               

                mdd_list.append(mdd)


                
                
                break #포문빠져나오기
                
                
                
            else:    
            
                asset = list1[i]*box["보유개수"] + rest
                profit = (list1[i]-box["보유가격"])*box["보유개수"]
                maximum = max(max(asset_list),asset)
                profit_percent = 100*(list1[i]/box["보유가격"] -1) #수익률
            
        
        asset_list.append(asset)
        profit_list.append(profit)
        maximum_list.append(maximum)
        profit_percent_list.append(profit_percent)
        
        
        if (asset - maximum) > 0:
            mdd = 0
        else:
            mdd = ((asset-maximum)/maximum)*100
            
            
        mdd_list.append(mdd)
        
            
    for k in range(len(list4)-i-1)  :
        
        asset = asset_list[-1]
        profit = profit_list[-1]
        maximum = maximum_list[-1]
        profit_percent = profit_percent_list[-1]
        mdd = mdd_list[-1]
        
        asset_list.append(asset)
        profit_list.append(profit)
        maximum_list.append(maximum)
        profit_percent_list.append(profit_percent)
        mdd_list.append(mdd)


        
        
        

        ########
        
    df['자산'] = asset_list
    df['전고점'] = maximum_list
    df['수익률'] = profit_percent_list
    df['수익금'] = profit_list
    df['MDD'] = mdd_list
    
    df['수익률'] = df['수익률']/100
    df['누적수익률'] = (df['수익률']+1).cumprod()-1
    df['수익률'] = df['수익률']*100
    df['누적수익률'] = df['누적수익률']*100

    trade_day = int(len(df.index))
    cagr = ((df['자산'][-1]/df['자산'][0])**(252/trade_day)-1)*100

    # 주요 지표 구하기
    backtest_list = {}
    backtest_list['Start date'] = start_date
    backtest_list['End date'] = end_date
    backtest_list['Name'] = code
    backtest_list['Start Money'] = start
    backtest_list['Total returns'] = str(round(df['누적수익률'][-1],2)) + "%"
    backtest_list['CAGR'] = str(round(cagr,2)) + "%"
    backtest_list['MDD'] = str(round(min(mdd_list),2)) + "%"
    backtest_list['Periods'] = trade_day

    #결과를 프린트하지말고 저장함
    result = pd.DataFrame(backtest_list,index=[1])
    result = result.transpose()
    result.columns = ['Algorithm1']
    
    
    #file_name1 = start_date + code + '.csv'
    
    #df.to_csv(file_name1)   #파일저장
    
    #file_name2 = start_date + code + '_result.csv'

    #result.to_csv(file_name2)
    
    return df, code

## 기간별 벡테스트 결과 함수

In [14]:
def period_result(total_reset1,total_reset2) :
     
    total_info1 = pd.DataFrame(total_reset1.sum(axis=1))
    total_info2 = pd.DataFrame(total_reset2.sum(axis=1))
    
    total_info_df = pd.concat([total_info1,total_info2], axis=1)
    
    total_info_df.columns = ['자산','수익금']
    
    
    total_info_df['수익률'] = (total_info_df['자산']/total_info_df['자산'][0] -1 )
    total_info_df['누적수익률'] = (total_info_df['수익률']+1).cumprod()-1
    total_info_df['수익률'] = total_info_df['수익률']*100
    total_info_df['누적수익률'] = total_info_df['누적수익률']*100
    
    # 전고점 mdd 
    list_period = list(total_info_df['자산'])
    
    maxlist = []
    
    for num , kes in enumerate(list_period) :
        
        if num == 0 :
            maxlist.append(i)
            
        else :
            maxlist.append(max(i,maxlist[-1]))
            
    total_info_df['전고점'] = maxlist
    
    total_info_df['MDD'] = ((total_info_df['자산'] - total_info_df['전고점'])/total_info_df['전고점'])*100

    
    
    trade_day = int(len(total_info_df.index))
    cagr = ((total_info_df['자산'][-1]/total_info_df['자산'][0])**(252/trade_day)-1)*100
    
    
    #주요 지표 구하기
    backtest_list = {}
    backtest_list['Start date'] = start_date
    backtest_list['End date'] = end_date
    backtest_list['Start Money'] = start_money
    backtest_list['Total returns'] = str(round(total_info_df['누적수익률'][-1],2)) + "%"
    backtest_list['CAGR'] = str(round(cagr,2)) + "%"
    backtest_list['MDD'] = str(round(min(list(total_info_df['MDD'])),2)) + "%"
    backtest_list['Periods'] = trade_day

    #결과를 프린트하지말고 저장함
    result = pd.DataFrame(backtest_list)
    result = result.transpose()
    #result.columns = ['Algorithm1']
    
    
    file_name1 = start_date[i] + 'total_info_df.csv'

    #total_df.to_csv(file_name1)

    file_name2 = start_date[i] + 'total_info_result.csv'

    #result.to_csv(file_name2)
    
    return total_info_df

## 전체 백테스트 결과 함수

In [15]:
def period_result2(kampa) :
             
    
    kampa['수익률'] = (kampa['자산']/kampa['자산'][0] -1 )
    kampa['누적수익률'] = (kampa['수익률']+1).cumprod()-1
    kampa['수익률'] = kampa['수익률']*100
    kampa['누적수익률'] = kampa['누적수익률']*100
    
    # 전고점 mdd 
    list_period2= list(kampa['자산'])
    
    maxlist2 = []
    
    for num , kes in enumerate(list_period2) :
        
        if num == 0 :
            maxlist2.append(kes)
            
        else :
            maxlist2.append(max(kes,maxlist2[-1]))
            
    kampa['전고점'] = maxlist2
    
    kampa['MDD'] = ((kampa['자산'] - kampa['전고점'])/kampa['전고점'])*100

    
    
    trade_day = int(len(kampa.index))
    cagr = ((kampa['자산'][-1]/kampa['자산'][0])**(252/trade_day)-1)*100
    
    
    #주요 지표 구하기
    backtest_list = {}
    backtest_list['Start date'] = '20180101'
    backtest_list['End date'] = '20210331'
    backtest_list['Start Money'] = 100000000
    backtest_list['Total returns'] = str(round(kampa['누적수익률'][-1],2)) + "%"
    backtest_list['CAGR'] = str(round(cagr,2)) + "%"
    backtest_list['MDD'] = str(round(min(list(kampa['MDD'])),2)) + "%"
    backtest_list['Periods'] = trade_day

    #결과를 프린트하지말고 저장함
    result = pd.DataFrame(backtest_list, index = [0])
    result = result.transpose()
    #result.columns = ['Algorithm1']
    
    
    file_name1 =  'total_df.csv'

    #total_df.to_csv(file_name1)

    file_name2 = 'total_result.csv'

    #result.to_csv(file_name2)
    
    total_df2 = kampa
    
    return total_df2

# 백테스팅 실행 : 총 2018년 1월 1일 부터 2020년 12월 30 까지 (나중에 3월 과 6월 데이터 추가하기) 총 13기간의 데이터 존재 , EX) 2017년 1월 1일 == 해당날짜에 딥러닝모델을 돌릴 수 있음.

In [98]:
#13개의 데이터 # 13개의 3개월 단위의 시간
datanumbers = 13 


start_date = [
    '20180101',
    '20180401',
    '20180701',
    '20181001',
    '20190101',
    '20190401',
    '20190701',
    '20191001',
    '20200101',
    '20200401',
    '20200701',
    '20201001',
    '20210101'
]

end_date = [
    '20180331',
    '20180630',
    '20180930',
    '20181231',
    '20190331',
    '20190630',
    '20190930',
    '20191231',
    '20200331',
    '20200630',
    '20200930',
    '20201231',
    '20210331'

]

#원래 처음 자본금 #시간이 지날수록 자산이 증가함

start_money = 10000000 

bula = []
boil = []

#백테스트를 위한 반복문
for i in tqdm(range(datanumbers)) : 
    
    #종목 코드 가져오기
    
    X_predict = X_data[200*i : 200*(i+1) ] #200개의 행씩 끊기
    
    X_predict1 = X_predict.dropna(how='any') #결측치 제거
    
    Y_predict = deep_model.predict(X_predict1.values[:,4:8]) # 딥러닝 모델을 통한 예측
    
    Y_predict1 = pd.DataFrame(Y_predict) #데이터 프레임화

    Y_predict2 = Y_predict1.sort_values(by=0,ascending=False).head(10) #확률이 높은 순대로 상위 10개 가져오기
    
    dir_list = np.array(Y_predict2.index) # 상위 10개 종목의 데이터 행위치 
    
    Code_list = np.array(X_predict1.index)[dir_list] #상위 10개 종목코드 리스트 #array 형태로 만들어야 저런 형식으로 슬라이싱가능
    
    Code_list = [i.replace("A","") for i in Code_list]
    
    for num,j in enumerate(Code_list) : #벡테스팅 돌리기
        
        #단일종목에 대한 백테스트 데이터
        df,code = backtest(start_date[i],end_date[i],j,start_money)  # start_date, end_date , code
        
        #단일종목의 데이터 가져오기
        if num == 0 :
            total_reset1 = pd.DataFrame(df['자산'])
            total_reset2 = pd.DataFrame(df['수익금'])
            
        
        else :
            total_reset1 = pd.concat([total_reset1,pd.DataFrame(df['자산'])],axis=1)
            total_reset2 = pd.concat([total_reset2,pd.DataFrame(df['수익금'])],axis=1)
            
        bula.append(code)    
        boil.append(df['자산'][-1] - df['자산'][0])    
        
            
        
    #분기별 13개의 벡테스트 결과데이터 출력
    period_result1 = period_result(total_reset1,total_reset2)
    
    start_money = period_result1['자산'][-1]/10
    
    if i == 0 :
        total_info1 = pd.DataFrame(period_result1['자산'])
        total_info2 = pd.DataFrame(period_result1['수익금'])
        
        
    else :
        total_info1 = pd.concat([total_info1,pd.DataFrame(period_result1['자산'])],axis=0)
        total_info2 = pd.concat([total_info2,pd.DataFrame(period_result1['수익금'])],axis=0)



total_df = pd.concat([total_info1,total_info2], axis=1)
    

total_df2 = period_result2(total_df)    
    
    
        
total_df2.to_csv("final_total_df.csv")        
        
    
    

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:15<00:00,  1.19s/it]


In [99]:
#df,code = backtest('20190401','20190630','218410',10000000)
#df

In [100]:
total_df2

,자산,수익금,수익률,누적수익률,전고점,MDD
날짜,,,,,,
2018-01-02,100000000.0,0.0,0.000000,0.000000,100000000.0,0.000000
2018-01-03,99719815.0,-280185.0,-0.280185,-0.280185,100000000.0,-0.280185
2018-01-04,98768691.0,-1231309.0,-1.231309,-1.508044,100000000.0,-1.231309
2018-01-05,99196440.0,-803560.0,-0.803560,-2.299486,100000000.0,-0.803560
2018-01-08,98576361.0,-1423639.0,-1.423639,-3.690389,100000000.0,-1.423639
...,...,...,...,...,...,...
2021-03-25,147109490.0,-2094356.0,47.109490,-91.032806,150116080.0,-2.002843
2021-03-26,146127890.0,-3075956.0,46.127890,-86.896429,150116080.0,-2.656737
2021-03-29,145978940.0,-3224906.0,45.978940,-80.871546,150116080.0,-2.755961


## 자산변화

In [101]:
px.line(total_df2, x=total_df2.index, y="자산",
        title=''' MACD 지표개선''')

In [30]:
#total_df = pd.read_csv("final_total_df.csv")
#total_df

## MDD 변화

In [48]:
px.line(total_df, x=total_df.index, y="MDD")

## 종목별 수익 보기

In [17]:
pd.set_option('display.max_rows', None)

profit_df = pd.DataFrame([bula,boil]).T

In [18]:
minus = profit_df[profit_df[1]<0]
plus = profit_df[profit_df[1]>0]
                 
print(len(plus),len(minus))
                 

72 40


In [19]:
profit_df


,0,1
0,218410,175507
1,002380,430192
2,241590,144550
3,003530,754600
4,000250,172200
5,018670,-220848
6,056190,652500
7,030190,54200
8,240810,0
9,032350,1496474


In [19]:
pd.set_option('display.max_rows', None)
